In [8]:
import numpy as np
from music21 import converter, instrument, stream, note, chord
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Carregar e pré-processar dados MIDI
def prepare_sequences(notes, n_vocab):
    sequence_length = 100

    # Mapear notas para números
    pitchnames = sorted(set(item for item in notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input, network_output = [], []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    network_input = network_input / float(n_vocab)

    network_output = np.array(network_output)

    return network_input, network_output

# Criar o modelo LSTM
def create_network(network_input, n_vocab):
    model = Sequential()
    model.add(LSTM(256, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True))
    model.add(LSTM(256))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(n_vocab, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

    return model

# Gerar novas melodias
def generate_notes(model, network_input, pitchnames, n_vocab):
    start = np.random.randint(0, len(network_input)-1)
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    for note_index in range(1000):  # Você pode ajustar o número de notas geradas
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern = np.append(pattern, index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

# Carregar dados MIDI
midi_path = './SweetDreams.mid'
midi_stream = converter.parse(midi_path)

notes = []
for element in midi_stream.flat:
    if isinstance(element, note.Note):
        notes.append(str(element.pitch))
    elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))

# Preparar sequências para o modelo
n_vocab = len(set(notes))
network_input, network_output = prepare_sequences(notes, n_vocab)

# Criar e treinar o modelo
model = create_network(network_input, n_vocab)
model.fit(network_input, network_output, epochs=50, batch_size=64)

# Obter os nomes das notas a partir dos dados MIDI
pitchnames = sorted(set(item for item in notes))

# Gerar novas melodias
generated_notes = generate_notes(model, network_input, pitchnames, n_vocab)

# Criar um stream MIDI a partir das notas geradas
offset = 0
output_notes = []

for pattern in generated_notes:
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    offset += 0.5  # Você pode ajustar o espaçamento entre as notas geradas

# Salvar as notas geradas em um novo arquivo MIDI
midi_stream_generated = stream.Stream(output_notes)
midi_stream_generated.write('midi', fp='output_generated.mid')



Epoch 1/50
28/28 [==============================] - 25s 529ms/step - loss: 3.1796
Epoch 2/50
28/28 [==============================] - 15s 534ms/step - loss: 2.9780
Epoch 3/50
28/28 [==============================] - 15s 535ms/step - loss: 2.9708
Epoch 4/50
28/28 [==============================] - 16s 570ms/step - loss: 2.9679
Epoch 5/50
28/28 [==============================] - 17s 592ms/step - loss: 2.9639
Epoch 6/50
11/28 [==========>...................] - ETA: 10s - loss: 2.9670